# Install necessary libraries

In [1]:
! pip install transformers accelerate bitsandbytes peft torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━

# Global Variables

# Global Functions

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. Define the correct local path for Version 2 (Note the '2' at the end)
LOCAL_MODEL_PATH = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"  

# 2. Define the 4-bit quantization config (Crucial for VRAM on a T4 GPU)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normalized Float 4-bit (recommended)
    bnb_4bit_compute_dtype=torch.bfloat16, # Use bfloat16 for computation if supported (faster)
)

In [3]:
# 3. Load Tokenizer and Model
# Note: You may need to add trust_remote_code=True for some models, but it is often unnecessary 
# for officially hosted models.
tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    LOCAL_MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto"
)

# Set the padding token, which is often missing or incorrectly set for Llama models
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Llama 3.1 8B Model loaded successfully!")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Llama 3.1 8B Model loaded successfully!


In [4]:
# Define the conversation history. We'll use a simple system message and a user query.
messages = [
    # Optional: A system message to define the model's persona or rules.
    {"role": "system", "content": "You are a concise, factual European news assistant."},
    # The user's question to test the model.
    {"role": "user", "content": "Explain why the Euro zone was created in one short paragraph."}
]

In [5]:
# Apply the template to create the final prompt string (input_ids)
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, # Tells the tokenizer to add the final 'assistant' header
    return_tensors="pt"
).to(model.device) # Move the prompt tokens to the GPU where the model is loaded

In [7]:
# The messages list is already defined from your previous cell
# messages = [...]

# 1. Apply the template to create the final prompt *string* (not a tensor yet)
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False, # Important: Returns a string
    add_generation_prompt=True
)

In [8]:
# 2. Tokenize the resulting string to get the required dictionary of tensors
# This is where the dictionary containing 'input_ids' and 'attention_mask' is created.
input_dict = tokenizer(prompt, return_tensors="pt").to(model.device)

In [9]:
import torch

with torch.no_grad():
    outputs = model.generate(
        **input_dict, # Pass the dictionary of tensors here
        max_new_tokens=256,    
        do_sample=True,        
        temperature=0.7        
    )

# Decode and print the output
response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract only the assistant response
assistant_response = response_text.split("assistant\n")[-1].strip()

print("\n--- ASSISTANT RESPONSE ONLY ---")
print(assistant_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- ASSISTANT RESPONSE ONLY ---
The Eurozone was created to facilitate economic integration among European countries. It began with the signing of the Maastricht Treaty in 1992, which established the European Monetary Union (EMU). The treaty aimed to create a single currency, the Euro, to promote economic unity, increase trade, and reduce transaction costs among participating countries. The Euro was introduced in 1999 and replaced the national currencies of participating countries in 2002.
